# Clasificación de Imágenes MRI con SVM + HOG

Este notebook implementa un pipeline clásico de visión computacional usando:

1. Extracción de características HOG (Histogram of Oriented Gradients)
2. Clasificación con SVM (Support Vector Machine)
3. Evaluación de resultados

In [ ]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import classification_report

# Configuración de rutas
DATASET_DIR = "../total/archive/Training"

In [ ]:
# Ejemplo de extracción de HOG y entrenamiento SVM
# features, labels = [], []
# for img_path in ...:
#     img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     hog_feat = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2))
#     features.append(hog_feat)
#     labels.append(label)
# clf = SVC(kernel='linear')
# clf.fit(features, labels)

Este pipeline es útil como línea base y para comparar con modelos de deep learning.